In [1]:
import pandas as pd
import torch
import os
import sys
import librosa
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import math

sys.path.append('../audio_preprocessing')
sys.path.append('../src')
sys.path.append('../model_training_utils')

import preprocessing_func_2
from gdsc_utils import PROJECT_DIR
import model_training

os.chdir(PROJECT_DIR)

In [2]:
df_metadata = pd.read_csv('data/metadata.csv')

df_metadata_class_1 = df_metadata[df_metadata["label"] == 1]
df_metadata_class_1.sample(n=6, replace=True)

,file_name,unique_file,path,species,label,subset,sample_rate,num_frames,length
2178,Aleetacurvicosta_GBIF3032104513_IN68465080_159...,Aleetacurvicosta_GBIF3032104513_IN68465080_159988,data/train/Aleetacurvicosta_GBIF3032104513_IN6...,Aleetacurvicosta,1,train,44100,458640,10.400000
1148,Aleetacurvicosta_GBIF1950026012_IN18986237_282...,Aleetacurvicosta_GBIF1950026012_IN18986237_28257,data/train/Aleetacurvicosta_GBIF1950026012_IN1...,Aleetacurvicosta,1,train,44100,598528,13.572063
1453,Aleetacurvicosta_GBIF2005290963_IN20215066_310...,Aleetacurvicosta_GBIF2005290963_IN20215066_31038,data/train/Aleetacurvicosta_GBIF2005290963_IN2...,Aleetacurvicosta,1,train,44100,451584,10.240000
311,Aleetacurvicosta_GBIF3039381926_IN68785661_161...,Aleetacurvicosta_GBIF3039381926_IN68785661_161247,data/val/Aleetacurvicosta_GBIF3039381926_IN687...,Aleetacurvicosta,1,validation,44100,807030,18.300000
492,Aleetacurvicosta_GBIF1990618565_IN19955380_307...,Aleetacurvicosta_GBIF1990618565_IN19955380_30728,data/val/Aleetacurvicosta_GBIF1990618565_IN199...,Aleetacurvicosta,1,validation,44100,437248,9.914921
1453,Aleetacurvicosta_GBIF2005290963_IN20215066_310...,Aleetacurvicosta_GBIF2005290963_IN20215066_31038,data/train/Aleetacurvicosta_GBIF2005290963_IN2...,Aleetacurvicosta,1,train,44100,451584,10.240000


In [3]:
df = pd.read_csv('data/big_data_processed_train_and_val.csv')
df.head()

,Unnamed: 0,file_path,label
0,0,data/big_data_processed_train_and_val/0.wav,56
1,1,data/big_data_processed_train_and_val/1.wav,56
2,2,data/big_data_processed_train_and_val/2.wav,56
3,3,data/big_data_processed_train_and_val/3.wav,56
4,4,data/big_data_processed_train_and_val/4.wav,56


In [4]:
for i in range(66):
    length = len(df[df["label"] == i])
    print(f"we are now in class {i} with {length} samples")

we are now in class 0 with 766 samples
we are now in class 1 with 81 samples
we are now in class 2 with 22 samples
we are now in class 3 with 54 samples
we are now in class 4 with 67 samples
we are now in class 5 with 301 samples
we are now in class 6 with 211 samples
we are now in class 7 with 317 samples
we are now in class 8 with 416 samples
we are now in class 9 with 395 samples
we are now in class 10 with 389 samples
we are now in class 11 with 490 samples
we are now in class 12 with 203 samples
we are now in class 13 with 190 samples
we are now in class 14 with 126 samples
we are now in class 15 with 87 samples
we are now in class 16 with 506 samples
we are now in class 17 with 1071 samples
we are now in class 18 with 45 samples
we are now in class 19 with 1217 samples
we are now in class 20 with 115 samples
we are now in class 21 with 753 samples
we are now in class 22 with 218 samples
we are now in class 23 with 145 samples
we are now in class 24 with 130 samples
we are now in 

In [5]:
df_test = df[df["label"] == 1].sample(frac=1).reset_index(drop=True)
print(len(df_test))

81


In [6]:
df_test_new = df_test.sample(n=6, replace=True)
df_test_new

,Unnamed: 0,file_path,label
53,2145,data/big_data_processed_train_and_val/2145.wav,1
52,21941,data/big_data_processed_train_and_val/21941.wav,1
51,21939,data/big_data_processed_train_and_val/21939.wav,1
71,14813,data/big_data_processed_train_and_val/14813.wav,1
9,19772,data/big_data_processed_train_and_val/19772.wav,1
80,7995,data/big_data_processed_train_and_val/7995.wav,1


In [7]:
def add_white_noise(x):
    rate = np.random.uniform(low=0.003, high=0.007)
    return x + rate*np.random.randn(len(x))

# data augmentation: shift sound in timeframe
def shift_sound(x):
    rate = np.random.randint(low=2, high=math.floor(len(x)/10000))
    return np.roll(x, int(len(x)//rate))

# data augmentation: stretch sound
def stretch_sound(x):
    rate = np.random.uniform(low=0.75, high=1.25)
    input_length = len(x)
    x = librosa.effects.time_stretch(x, rate=rate)
    if len(x)>input_length:
        return x[:input_length]
    else:
        return np.pad(x, (0, max(0, input_length - len(x))), "constant")

def change_pitch(x):
    pitch_factor = 0
    while pitch_factor == 0:
        pitch_factor = np.random.randint(low=-4, high=4)
    return librosa.effects.pitch_shift(x, sr=44100, n_steps=pitch_factor)
    
def combined_sound(x):
    return add_white_noise(shift_sound(stretch_sound(change_pitch(x))))


In [8]:
path = "data/big_data_processed_train_and_val/13780.wav"
wav = preprocessing_func_2.load_wav(path=path)

wav_white_noise = add_white_noise(wav)
wav_shifted = shift_sound(wav)
wav_stretched = stretch_sound(wav)
wav_pitch_changed = change_pitch(wav)
wav_combined = combined_sound(wav)

In [9]:
import IPython

IPython.display.Audio(wav, rate=44100)

In [10]:
IPython.display.Audio(wav_white_noise, rate=44100)

In [11]:
IPython.display.Audio(wav_shifted, rate=44100)

In [12]:
IPython.display.Audio(wav_stretched, rate=44100)

In [13]:
IPython.display.Audio(wav_pitch_changed, rate=44100)

In [14]:
IPython.display.Audio(wav_combined, rate=44100)

In [15]:
import soundfile as sf

my_argumentations = [add_white_noise, shift_sound, stretch_sound, change_pitch, combined_sound]

def upsample_and_save(df, argumentations=my_argumentations, max_num_of_data_per_class:int=1350, sr:int=44100):
    new_paths_steam = "data/big_data_upsample_train_and_val"
    new_paths = []
    new_labels = []
    i = 0
    for class_num in range(66):
        df_current_class = df[df["label"] == class_num].sample(frac=1).reset_index(drop=True)
        num_of_samples:int = len(df_current_class)
        if num_of_samples >= max_num_of_data_per_class:
            continue
        num_of_upsample_to_do:int = max_num_of_data_per_class - num_of_samples
        num_of_upsample_done:int = 0
        while num_of_upsample_done < num_of_upsample_to_do:
            paths, labels = list(df_current_class["file_path"]), list(df_current_class["label"])
            for (path, label) in zip(paths, labels):
                wav = preprocessing_func_2.load_wav(path=path)
                for argumentation in argumentations:
                    modified_wav = argumentation(wav)
                    sf.write(f"{new_paths_steam}/{i}.wav", modified_wav, sr)
                    new_paths.append(f"{new_paths_steam}/{i}.wav")
                    new_labels.append(label)
                    num_of_upsample_done += 1
                    i += 1
                if num_of_upsample_done >= num_of_upsample_to_do:
                    break
    return new_paths, new_labels

In [16]:
new_paths, new_labels = upsample_and_save(df)

In [17]:
my_dict = {
    "file_path": new_paths,
    "label": new_labels
}

processed_data_df = pd.DataFrame(my_dict)
processed_data_df.to_csv("data/big_argumentation_data_train_and_val.csv")